In [1]:
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re

# from collections import defaultdict
import pandas as pd
from collections import Counter, defaultdict
# import numpy as np
# #import xmltodict
# import networkx as nx
# from sqlalchemy import text

repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)
    
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import json
import gzip

import republic.model.republic_document_model as rdm
resolutions_file = os.path.join("/Users/rikhoekstra/surfdrive/Republic/gekoppelde_resources/", "rampjaar-ordinaris-resolutions.json.gz")


# resolutions_file = "../../data/resolutions/rampjaar-ordinaris-resolutions.json.gz"

# Reading the JSON representations from file again and turning 
# them into Resolution objects again
with gzip.open(resolutions_file, 'rt') as fh:
    # the document model has a convenience function to turn a JSON representation
    # to a Resolution object: json_to_republic_resolution
    resolutions_1672 = [rdm.json_to_republic_resolution(res) for res in json.load(fh)]

    
# or    
# query = {
#     "bool": {
#         "must": [
#             {"match": {"type": "resolution"}},
#             {"match": {"metadata.session_year": 1672}}
#         ]
#     }
# }

# resolutions_1672 = rep_es.retrieve_resolutions_by_query(query, size=10000)

In [3]:
from republic.extraction import template_detection

In [122]:
sr = template_detection.SearchResolutions(searchterms=['ontfangen'], resolutions=resolutions_1672)

In [123]:
lr2 = sr.search_analitic(bdir='/Users/rikhoekstra/develop/analiticcl/')

In [ ]:
lr1


In [ ]:
lr2

In [8]:
import langid
def resolutions2kwic(resolutions, result, searchterms=['ontfangen'], langids=['nl'], context_size=5):
    xresult = []
    for rslt in result: #filter_result:
        r = [r for r in resolutions if r.id==rslt][0]
        id = r.metadata['id']
        text = '\n'.join([p.text for p in r.paragraphs if langid.classify(p.text)[0] in langids])
        s = result[id]
        for v in s:
            offset = v['offset']
            variant = max(v['variants'], key=lambda x: x['dist_score'])
            fnd = variant['text']
            kwc = template_detection.get_keyword_context(text.lower(),
                                      fnd,
                                      context_size=context_size)
            for item in kwc:
                context = f"{item['prefix']} {item['keyword']} {item['suffix']}"
                xresult.append((id, v['input'], fnd, context, item['prefix'], item['suffix']))
    return xresult

In [131]:
resolutions2kwic(resolutions_1672, lr)

IndexError: list index out of range

In [128]:
ddf = template_detection.StratifiedPhrase(resolutions=resolutions_1672, searchresult=lr1, searchterms=['ontfangen'], 
                                          langids=["nl"], context_size=10, 
                                          prefix_size=5, suffix_size=5)

In [129]:
ddf.resolutions2kwic()

IndexError: list index out of range

In [130]:
ddf.res2frame()

IndexError: list index out of range

In [15]:
ddf.df

,resolutie,keyword,found,context,prefix,suffix
0,session-1672-01-07-num-1-resolution-2,Ontfangen,ontfangen,ontfangen een missive van het collegie,,een missive van het collegie
1,session-1672-01-07-num-1-resolution-4,Ontfangen,ontfangen,ontfangen een missive vanden heer ambassadeur,,een missive vanden heer ambassadeur
2,session-1672-01-07-num-1-resolution-4,Ontfangen,ontfangen,geen resolutie en is gevallen. ontfangen twee ...,geen resolutie en is gevallen.,twee missiven vanden heer ambassadeur
3,session-1672-01-07-num-1-resolution-4,Ontfangen,ontfangen,ontfangen een missive vanden heer ambassadeur,,een missive vanden heer ambassadeur
4,session-1672-01-07-num-1-resolution-4,Ontfangen,ontfangen,geen resolutie en is gevallen. ontfangen twee ...,geen resolutie en is gevallen.,twee missiven vanden heer ambassadeur
...,...,...,...,...,...,...
3153,session-1672-12-29-num-1-resolution-7,Ontfanger,ontfangen,waerop geen resolutie is gevallen. ontfangen t...,waerop geen resolutie is gevallen.,twee missiven vanden heer van
3154,session-1672-12-29-num-1-resolution-7,Ontfanger,ontfangen,waerop geen resolutie is gevallen. ontfangen e...,waerop geen resolutie is gevallen.,een missive vanden soon vanden
3155,session-1672-12-29-num-1-resolution-7,Ontfanger,ontfangen,waerop geen resolutie is gevallen. ontfangen e...,waerop geen resolutie is gevallen.,"een missive vanden resident sasburgh,"
3156,session-1672-12-29-num-1-resolution-7,Ontfanger,ontfangen,waerop geen resolutie is gevallen. ontfangen e...,waerop geen resolutie is gevallen.,een missive van mr. johan


In [24]:
flds = list(range(ddf.context_size))
ddf.suffix_size
# ddf.df[flds] = ddf.df['suffix'].str.split(' ', expand=True)
# for c in flds:
#     print(c)
#     ddf.df[str(c)+'_norm'] = ddf.df[c].apply(lambda x:normalize(x,rev_kws=rev_kws))
#         strati_df = self.df.groupby(flds).agg({"resolutie":'count'}).sort_values('resolutie',ascending=False)
#         return strati_df

5

In [28]:
ddf.stratified_df('suffix', norms=template_detection.rev_kws)

resolutie
0    1       2      3         4                      
een  missive van    het       collegie            174
             vanden heer      van                 164
             vant   collegie  ter                 101
             vande  heeren    van                  97
             vanden heer      brasser,             74
...                                               ...
                    baron     van                   1
                    bailliu,, ende                  1
                    bailliu,  n                     1
                              burgemeester          1
zijn bij     sij    mon       ribeiro               1

[728 rows x 1 columns]

In [30]:
what='suffix'
flds=list(range(ddf.suffix_size))
ddf.df[flds]=ddf.df[what].str.split(' ', expand=True)#stratified_df('suffix', norms=template_detection.kws)

In [31]:
for c in flds:
    ddf.df[str(c)+'_norm'] = ddf.df[c].apply(lambda x: template_detection.normalize(x, rev_kws=template_detection.rev_kws))

In [32]:
ddf.df.groupby(flds)

In [33]:
from settings import inventory_metadata_file
inventory_metadata_file

'./data/inventories/inventory_metadata.json'

In [34]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

inv_num = 3760
# inv_config = set_config_inventory_num(inv_num,ocr_type="pagexml")

rep_es = initialize_es(host_type="external")

rep_es

In [35]:
with open("/Users/rikhoekstra/develop/republic_clean/data/inventories/inventory_metadata.json", 'rt') as fn:
    imd = json.load(fn)

In [36]:
[i for i in imd if i.get('inventory_num')==3760][0]['year']

1705

In [37]:
resolutions = []
for ri, res in enumerate(rep_es.scroll_inventory_resolutions(3760)):
    resolutions.append(res)
    if len(resolutions) == 200:
        break

total hits: {'value': 7088, 'relation': 'eq'} 	hits per scroll: 10


In [38]:
len(resolutions)

200

In [39]:
query = {
    "bool": {
        "must": [
            {"match": {"type": "resolution"}},
            {"match": {"metadata.session_year": 1705}}
        ]
    }
}

resolutions_1705 = rep_es.retrieve_resolutions_by_query(query, size=10000)



In [159]:
sr = template_detection.SearchResolutions(searchterms=dels1705, resolutions=resolutions_1705)

In [160]:
lr = sr.search_analitic(bdir='/Users/rikhoekstra/develop/analiticcl/')

In [161]:
len(lr)


1941

In [170]:
template_detection

<module 'republic.extraction.template_detection' from '/Users/rikhoekstra/develop/republic_clean/republic/extraction/template_detection.py'>

In [172]:
# def resolutions2kwic(resolutions, result, searchterms=['ontfangen'], langids=['nl'], context_size=5):
xresult = []
langids=ddf.langids
context_size = 10
for rslt in list(lr.keys()): #filter_result:
    r = [r for r in resolutions_1705 if r.id==rslt][0]
    rid = r.metadata['id']
    text = '\n'.join([p.text for p in r.paragraphs if langid.classify(p.text)[0] in langids])
    s = lr[rid]
    for v in s:
        offset = v['offset']
        variant = max(v['variants'], key=lambda x: x['dist_score'])
        fnd = variant['text']
        kwc = template_detection.get_keyword_context(text.lower(),
                                  fnd.lower(),
                                  context_size=context_size)
        for item in kwc:
            context = f"{item['prefix']} {item['keyword']} {item['suffix']}"
            xresult.append((id, v['input'], fnd, context, item['prefix'], item['suffix']))


In [173]:
len(xresult)

2972

In [175]:
tdf = pd.DataFrame(xresult, columns=['resolutie', 'keyword', 'found', 'context', 'prefix', 'suffix'])

In [184]:
flds = list(range(10))
tdf[flds] = tdf['suffix'].str.split(' ', expand=True)

In [188]:
tdf.groupby(['keyword']+flds).agg({"resolutie":'count'}).sort_values('resolutie',ascending=False)

resolutie
keyword      0  1           2      3          4              5         6             7             8      9                         
Ham          ,  ende        andere haer       hoogh          mogende   gedeputeerden tot           de     militaire              362
Vrybergen    ,  haer        hoogh  mogende    extraordinaris envoyé    aen           het           hof    van                    224
Broeckhuysen ,  ende        andere haer       hoogh          mogende   gedeputeerden tot           de     militaire              141
Gent         ,  ende        andere haer       hoogh          mogende   gedeputeerden tot           de     saecken                123
van Essen    ,  ende        andere haer       hoogh          mogende   gedeputeerden tot           de     saecken                107
...                                                                                                                              ...
Heemskerck   ,  gedeputeert wegens haer       hoogh          mogende   te            velde         ,      is                       1
                ende        van    schagen,   haer           hoogh     mogende       gedeputeerden te     velde                    1
                een         van    haer       hoogh          mogende   gedeputeerde  te            velde, van                      1
                de          welcke van        wegende        provincie van           hollandt      ende   west-vrieslandt          1
Steenbergen  te mogen       doen   uytvoeren, vry,           ende      sonder        bealinge      van    's                       1

[765 rows x 1 columns]

In [187]:
tdf.keyword

0       van Lintelo
1         Vrybergen
2              Gent
3              Gent
4              Gent
           ...     
2967      Vrybergen
2968      Vrybergen
2969       Tulleken
2970     Heemskerck
2971     Heemskerek
Name: keyword, Length: 2972, dtype: object

In [149]:
template_detection.get_keyword_context(text.lower(), fnd.lower())

[]

In [118]:
for item in list(lr.keys())[0]:
    rslt=lr[item]
    r = [r for r in resolutions_1705 if r.id==rslt]
    rslt

In [193]:
ddf = template_detection.StratifiedPhrase(resolutions=resolutions_1705, 
                                          searchresult=lr, 
                                          langids=['nl'], 
                                          searchterms=dels1705, 
                                           context_size=10, 
                                          prefix_size=5, suffix_size=5)

In [195]:
ddf.res2frame()

In [196]:
ddf.df

,resolutie,keyword,found,context,prefix,suffix
0,session-1705-07-06-num-1-resolution-8,van Lintelo,van Lintelo,"den secretaris van den heere van lintelo , hae...",den secretaris van den heere,", haer ho, mog. extraordinaris"
1,session-1705-07-06-num-1-resolution-9,Vrybergen,Vrybergen,"secretaris wan den heere van vrybergen , haer ...",secretaris wan den heere van,", haer hoogh mogende extraordinaris"
2,session-1705-07-06-num-1-resolution-21,Gent,Gent,"van jacobus danielsz, koopman tot gent ; ls na...","van jacobus danielsz, koopman tot",; ls naer voorgriende deliberatie
3,session-1705-07-06-num-1-resolution-21,Gent,Gent,"straetsteenen, platielbackers en volaerde, van...","straetsteenen, platielbackers en volaerde, van",na dese landen te mogen
4,session-1705-07-06-num-1-resolution-21,Gent,Gent,en weder in retour na gent uytvoeren gepermitt...,en weder in retour na,"uytvoeren gepermitteerde waren, uytgesonderdt ..."
...,...,...,...,...,...,...
2967,session-1705-05-02-num-1-resolution-24,Vrybergen,Vrybergen,hedges met hem heere van vrybergen over die sa...,hedges met hem heere van,"over die saecke spreeckende, hem"
2968,session-1705-05-02-num-1-resolution-24,Vrybergen,Vrybergen,"hadde, en hem heer van vrybergen derhalven had...","hadde, en hem heer van",derhalven hadde versocht aen haer
2969,session-1705-05-02-num-1-resolution-24,Tulleken,Tulleken,"in handen van de heeren tulleken , ende andere...",in handen van de heeren,", ende andere haer hoogh"
2970,session-1705-05-02-num-1-resolution-34,Heemskerck,Heemskerck,"en genomineert den heer van heemskerck , om by...",en genomineert den heer van,", om by haer hoogh"


[{'p': 0,
  'input': 'Vrybergen',
  'offset': {'begin': 2008, 'end': 2017},
  'variants': [{'text': 'Vrybergen',
    'score': 1.0,
    'dist_score': 1.0,
    'freq_score': 1.0,
    'lexicons': ['/tmp/analit_search.tsv']}]},
 {'p': 0,
  'input': 'Ham',
  'offset': {'begin': 2409, 'end': 2412},
  'variants': [{'text': 'Ham',
    'score': 1.0,
    'dist_score': 1.0,
    'freq_score': 1.0,
    'lexicons': ['/tmp/analit_search.tsv']}]}]

In [234]:
tdf

,resolutie,keyword,found,context,prefix,suffix,0,1,2,3,4,0_norm,1_norm,2_norm,3_norm,4_norm
178,session-1705-01-01-num-1-resolution-22,Ham,Ham,"in handen van de heeren ham , en andere haer h...",in handen van de heeren,", en andere haer hoogh",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
180,session-1705-01-01-num-1-resolution-23,Ham,Ham,"in handen van de heeren ham , en andere haer h...",in handen van de heeren,", en andere haer hoogh",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
186,session-1705-01-01-num-1-resolution-32,Ham,Ham,"in handen van de heeren ham , en andere haer h...",in handen van de heeren,", en andere haer hoogh",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
188,session-1705-01-03-num-1-resolution-11,Ham,Ham,"in handen van de heeren ham , en andere haer h...",in handen van de heeren,", en andere haer hoogh",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
197,session-1705-01-05-num-1-resolution-15,Ham,Ham,"in handen van de heeren ham , en andere haer h...",in handen van de heeren,", en andere haer hoogh",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2940,session-1705-04-30-num-1-resolution-17,Ham,Ham,"in handen van de heeren ham , ende andere haer...",in handen van de heeren,", ende andere haer hoogh",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
2941,session-1705-04-30-num-1-resolution-18,Ham,Ham,"in handen van de heeren ham , ende andere haer...",in handen van de heeren,", ende andere haer hoogh",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
2942,session-1705-04-30-num-1-resolution-19,Ham,Ham,"in handen van de heeren ham , ende ende andere...",in handen van de heeren,", ende ende andere haer",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer
2943,session-1705-04-30-num-1-resolution-20,Ham,Ham,"in handen van de heeren ham , ende andere haer...",in handen van de heeren,", ende andere haer hoog",in,handen,van,de,heeren,<in>,<handen>,<van>,lidwoord,heer


In [245]:
tdf = ddf.df[(ddf.df.keyword=='Ham') &
      (ddf.df['context'].str.contains("handen"))]
tdf.groupby([0,1,2,3,4]).agg({'resolutie':'count',
                                                        'prefix':lambda x: x.iloc[0],
                                                       'suffix':lambda x: x.iloc[0]})

resolutie  \
0      1       2      3            4                   
handen van     -      de           heeren          1   
               de     bovengemelde heeren          1   
                      heeren       van             4   
in     'handen van    de           heeren          1   
       handen  van    de           heeren        311   
                      del          heeren          1   
               yan    de           heeren          1   
in.    handen  van    de           heeren          1   
inw    handen  van    de           heeren          1   
werden in      handen vande        heeren          2   

                                                                      prefix  \
0      1       2      3            4                                           
handen van     -      de           heeren             handen van - de heeren   
               de     bovengemelde heeren  handen van de bovengemelde heeren   
                      heeren       van              handen van de heeren van   
in     'handen van    de           heeren           in 'handen van de heeren   
       handen  van    de           heeren            in handen van de heeren   
                      del          heeren           in handen van del heeren   
               yan    de           heeren            in handen yan de heeren   
in.    handen  van    de           heeren           in. handen van de heeren   
inw    handen  van    de           heeren           inw handen van de heeren   
werden in      handen vande        heeren      werden in handen vande heeren   

                                                                     suffix  
0      1       2      3            4                                         
handen van     -      de           heeren          , ende andere haer hoogh  
               de     bovengemelde heeren         , ende andere haer hooogh  
                      heeren       van             , ende andere haer hoogh  
in     'handen van    de           heeren          , ende andere haer hoogh  
       handen  van    de           heeren            , en andere haer hoogh  
                      del          heeren          , ende andere haer hoogh  
               yan    de           heeren          , ende andere haer hoogh  
in.    handen  van    de           heeren  , ende andere-haer hoogh mogende  
inw    handen  van    de           heeren          , ende andere haer hoogh  
werden in      handen vande        heeren            , en andere haer hoogh

In [244]:
tdf = ddf.df[(ddf.df.keyword=='Ham') &
      (ddf.df['context'].str.contains("handen"))]
tdf.groupby(['0_norm','1_norm','2_norm', '3_norm','4_norm']).agg({'resolutie':'count',
                                                        'prefix':lambda x: x.iloc[0],
                                                       'suffix':lambda x: x.iloc[0]})

resolutie  \
0_norm   1_norm    2_norm   3_norm         4_norm              
<handen> <van>     <->      lidwoord       heer            1   
                   lidwoord <bovengemelde> heer            1   
                            heer           <van>           4   
<in>     <'handen> <van>    lidwoord       heer            1   
         <handen>  <van>    <del>          heer            1   
                            lidwoord       heer          312   
                   <yan>    lidwoord       heer            1   
<inw>    <handen>  <van>    lidwoord       heer            1   
<werden> <in>      <handen> van            heer            2   

                                                                              prefix  \
0_norm   1_norm    2_norm   3_norm         4_norm                                      
<handen> <van>     <->      lidwoord       heer               handen van - de heeren   
                   lidwoord <bovengemelde> heer    handen van de bovengemelde heeren   
                            heer           <van>            handen van de heeren van   
<in>     <'handen> <van>    lidwoord       heer             in 'handen van de heeren   
         <handen>  <van>    <del>          heer             in handen van del heeren   
                            lidwoord       heer              in handen van de heeren   
                   <yan>    lidwoord       heer              in handen yan de heeren   
<inw>    <handen>  <van>    lidwoord       heer             inw handen van de heeren   
<werden> <in>      <handen> van            heer        werden in handen vande heeren   

                                                                      suffix  
0_norm   1_norm    2_norm   3_norm         4_norm                             
<handen> <van>     <->      lidwoord       heer     , ende andere haer hoogh  
                   lidwoord <bovengemelde> heer    , ende andere haer hooogh  
                            heer           <van>    , ende andere haer hoogh  
<in>     <'handen> <van>    lidwoord       heer     , ende andere haer hoogh  
         <handen>  <van>    <del>          heer     , ende andere haer hoogh  
                            lidwoord       heer       , en andere haer hoogh  
                   <yan>    lidwoord       heer     , ende andere haer hoogh  
<inw>    <handen>  <van>    lidwoord       heer     , ende andere haer hoogh  
<werden> <in>      <handen> van            heer       , en andere haer hoogh

In [241]:
tdf = ddf.df[(ddf.df.keyword=='Ham') &
      (ddf.df['context'].str.contains("handen"))]
tdf.groupby(['4_norm','3_norm','2_norm','1_norm']).agg({'resolutie':'count',
                                                        'prefix':lambda x: x.iloc[0],
                                                       'suffix':lambda x: x.iloc[0]})

resolutie  \
4_norm 3_norm         2_norm   1_norm                 
<van>  heer           lidwoord <van>              4   
heer   <bovengemelde> lidwoord <van>              1   
       <del>          <van>    <handen>           1   
       lidwoord       <->      <van>              1   
                      <van>    <'handen>          1   
                               <handen>         313   
                      <yan>    <handen>           1   
       van            <handen> <in>               2   

                                                                     prefix  \
4_norm 3_norm         2_norm   1_norm                                         
<van>  heer           lidwoord <van>               handen van de heeren van   
heer   <bovengemelde> lidwoord <van>      handen van de bovengemelde heeren   
       <del>          <van>    <handen>            in handen van del heeren   
       lidwoord       <->      <van>                 handen van - de heeren   
                      <van>    <'handen>           in 'handen van de heeren   
                               <handen>             in handen van de heeren   
                      <yan>    <handen>             in handen yan de heeren   
       van            <handen> <in>           werden in handen vande heeren   

                                                             suffix  
4_norm 3_norm         2_norm   1_norm                                
<van>  heer           lidwoord <van>       , ende andere haer hoogh  
heer   <bovengemelde> lidwoord <van>      , ende andere haer hooogh  
       <del>          <van>    <handen>    , ende andere haer hoogh  
       lidwoord       <->      <van>       , ende andere haer hoogh  
                      <van>    <'handen>   , ende andere haer hoogh  
                               <handen>      , en andere haer hoogh  
                      <yan>    <handen>    , ende andere haer hoogh  
       van            <handen> <in>          , en andere haer hoogh

In [198]:
ddf.stratified_df('prefix',  norms=template_detection.rev_kws, with_kw=True)

resolutie
keyword      0           1           2         3           4                   
Ham          in          handen      van       de          heeren           311
Gent         handen      van         de        heeren      van              167
van Essen    in          handen      van       de          heeren           159
Broeckhuysen handen      van         de        heeren      van              100
Vrybergen    missive     van         den       heere       van              100
...                                                                         ...
Kniphuisen   volle       neeve,      ferdinant baron       van                1
             van         georg       wilhelm,  grave       van                1
Heemskerek   en          genomineert den       heer        van                1
Heemskerck   waergenomen by          den       heere       van                1
Tulleken     aengestelt, voor        de        infanterye, ripperda,          1

[667 rows x 1 columns]

In [47]:
basedir = '/Users/rikhoekstra/republic_out/20211115/'

In [48]:
lst = os.listdir(basedir)

In [49]:
allspans = []
years = {}
for y in lst:
    with open(os.path.join(basedir, y)) as fn:
        try:
            j = json.load(fn)
            plists = [r['spans'] for r in j]
            yearspans = [s for r in plists for s in r if s['class'] in ['president', 'delegate']]
            yearpres = [s for r in plists for s in r if s['class'] in ['president']]
            years[y] = {'presidents':yearpres, 'delegates':yearspans}
            allspans.extend(yearspans)
        except json.JSONDecodeError:
            print (y)

text.json


In [50]:
y1705 = years.get('1705_out.json')

In [51]:
countyears = {}
for y in lst:
    with open(os.path.join(basedir, y)) as fn:
        try:
            j = json.load(fn)
            plists = [r['spans'] for r in j]
            yearspans = [s for r in plists for s in r if s['class'] in ['president', 'delegate']]
            yearpres = [s for r in plists for s in r if s['class'] in ['president']]
            countyears[y] = {'presidents':len(yearpres), 'delegates':len(yearspans)-len(yearpres)}
            allspans.extend(yearspans)
        except json.JSONDecodeError:
            print (y)

text.json


In [52]:
countyears['1705_out.json']

{'presidents': 288, 'delegates': 4343}

In [53]:
len([span for span in allspans if span['pattern'].find('Essen')>-1])

7274

In [54]:
from collections import Counter

In [55]:
len(allspans)

758170

In [56]:
sum([countyears[y]['presidents'] for y in years])

20378

In [57]:
sum([countyears[y]['delegates'] for y in years])

358707

In [58]:
from collections import defaultdict

In [59]:
president_variants = defaultdict(list)
delegate_variants = defaultdict(list)
for y in years:
    for i in years[y]['presidents']:
        president_variants[f"{i['delegate_name']} - {i['delegate_id']}"].append(i['pattern'])
    for i in years[y]['delegates']:
        delegate_variants[f"{i['delegate_name']} - {i['delegate_id']}"].append(i['pattern'])

In [60]:
delegate_variants2 = {}
for key in delegate_variants:
    delegate_variants2[key] = Counter([variant for variant in delegate_variants[key]])

In [61]:
delegate_variants_year = {}
for y in years:
    delegate_variants_year[y] = defaultdict(list)
    thisyear = delegate_variants_year[y]
#     for i in years[y]['presidents']:
#         president_variants[f"{i['delegate_name']} - {i['delegate_id']}"].append(i['pattern'])
    for i in years[y]['delegates']:
        thisyear[f"{i['delegate_name']} - {i['delegate_id']}"].append(i['pattern'])

- varianten per jaar
- verschillende varianten per jaar
- verwijzing naar zittingsdagen
- deduplicate en disambiguate
- variants query?

In [62]:
from republic.data.delegate_database import abbreviated_delegates

In [63]:
variants_year = {}
for y in delegate_variants_year:
    thisyear = delegate_variants_year[y]
    variants_year[y] = {}
    for delegate in thisyear:
        thisdelegate = variants_year[y]
        variant_c = Counter([variant for variant in thisyear[delegate]])
        variants_year[y][delegate] = {}
        variants_year[y][delegate]['main_variant'] = variant_c.most_common(1)[0][0]
        variants_year[y][delegate]['variants'] = list(set(thisyear[delegate]))
        


In [64]:
variants_year['1705_out.json']

{'Essenius - 432': {'main_variant': 'van Essen',
  'variants': ['van Essen',
   'van Yssenm',
   'van sssen',
   'van Tisen',
   'van Issem',
   'van TYssem',
   'van Esen',
   'vonEssen',
   'vanTssem',
   'vanTissen',
   'Tssen',
   "'van Esten",
   'van Jssem',
   'van Tsem',
   'van Ten',
   'van Tssen',
   'Essen',
   'vinGent',
   'van sssem',
   'van IHIjssen',
   'van issen',
   'wan Essen',
   'van rsfem',
   'van Tssem',
   'van Yssen',
   'vanGent',
   'van rssem',
   'van ssen',
   'van Tisem',
   'van Issen',
   'van tssen',
   'van Yssem',
   'van Gent',
   'vanTssen',
   'vanEsen',
   'van risem',
   'wan Tssen',
   'wan: Essen',
   'vanEssen']},
 'Gockinga - 17325': {'main_variant': 'Gockinga',
  'variants': ['Nilandr Gockinga',
   'Golsinga',
   'Goccinga',
   'Goslinga',
   'Gockinga:',
   'Gockunga',
   'Gockiuga',
   'Gockinga',
   'Colfsinga',
   'OColsinga',
   'Nilande Gockinga']},
 'du Tour van Warmenhuisen - 12373': {'main_variant': 'du Tour',
  'variants': ['D

In [65]:
dels1705 = [v['main_variant'] for k, v in variants_year['1705_out.json'].items()]